# FSL Wrapper Tutorial

This notebook demonstrates how to use the FSL Wrapper modules in the medical project for neuroimaging processing with FSL (FMRIB Software Library).

## Overview

The FSL Wrapper module provides Python interfaces for:
1. **BET (Brain Extraction Tool)**: Brain tissue extraction from structural images
2. **FLIRT (Linear Registration Tool)**: Linear image registration and transformation
3. **FAST (Segmentation Tool)**: Automated tissue-type segmentation
4. **FSLMaths**: Mathematical operations on neuroimages
5. **Pipeline Processing**: Build complex multi-step processing workflows

## Table of Contents
1. [Setup and Environment Check](#setup)
2. [Brain Extraction with BET](#bet)
3. [Image Registration with FLIRT](#flirt)
4. [Tissue Segmentation with FAST](#fast)
5. [Mathematical Operations with FSLMaths](#fslmaths)
6. [Pipeline Processing](#pipeline)
7. [Advanced Usage](#advanced)


## 1. Setup and Environment Check {#setup}


In [ ]:
# Check if the FSL Wrapper modules exist and are accessible
import os
import sys
import warnings
from pathlib import Path
import numpy as np

# Suppress warnings for cleaner output
warnings.filterwarnings("ignore")

# Get the current notebook directory
notebook_dir = Path.cwd()
project_root = notebook_dir.parent  # Assuming notebook is in tutorial/
fslwrapper_path = project_root / "utils" / "fslwrapper"

print(f"Notebook directory: {notebook_dir}")
print(f"Project root: {project_root}")
print(f"FSL Wrapper path: {fslwrapper_path}")
print(f"FSL Wrapper path exists: {fslwrapper_path.exists()}")

if fslwrapper_path.exists():
    print("FSL Wrapper modules found!")
    
    # Check available files
    fslwrapper_src = fslwrapper_path / "fslwrapper"
    files = {
        "__init__.py": fslwrapper_src / "__init__.py",
        "core.py": fslwrapper_src / "core.py",
        "bet.py": fslwrapper_src / "bet.py",
        "flirt.py": fslwrapper_src / "flirt.py",
        "fast.py": fslwrapper_src / "fast.py",
        "math.py": fslwrapper_src / "math.py",
        "utils.py": fslwrapper_src / "utils.py",
        "exceptions.py": fslwrapper_src / "exceptions.py"
    }
    
    print("\nAvailable files:")
    for file_name, file_path in files.items():
        status = "✓" if file_path.exists() else "✗"
        print(f"  {status} {file_name}")
        
else:
    print("FSL Wrapper modules not found!")

# Add FSL Wrapper to Python path
if str(fslwrapper_path) not in sys.path:
    sys.path.insert(0, str(fslwrapper_path))
    print(f"\nPython path updated with: {fslwrapper_path}")


In [ ]:
def test_fslwrapper_imports():
    """Test importing FSL Wrapper modules"""
    print("Testing FSL Wrapper Module Imports:")
    print("-" * 40)
    
    modules_to_test = [
        ("fslwrapper", "FSLWrapper", "Main Wrapper"),
        ("fslwrapper", "BET", "Brain Extraction"),
        ("fslwrapper", "FLIRT", "Registration"),
        ("fslwrapper", "FAST", "Segmentation"),
        ("fslwrapper", "FSLMaths", "Math Operations"),
        ("fslwrapper", "FSLPipeline", "Pipeline Processing"),
    ]
    
    import_results = {}
    imported_classes = {}
    
    for module_name, class_name, display_name in modules_to_test:
        try:
            module = __import__(module_name, fromlist=[class_name])
            cls = getattr(module, class_name)
            import_results[display_name] = True
            imported_classes[class_name] = cls
            print(f"{display_name:20}: ✓ Successfully imported {class_name}")
        except ImportError as e:
            import_results[display_name] = False
            print(f"{display_name:20}: ✗ Import failed - {str(e)[:50]}...")
        except AttributeError as e:
            import_results[display_name] = False
            print(f"{display_name:20}: ✗ Class not found - {str(e)[:50]}...")
        except Exception as e:
            import_results[display_name] = False
            print(f"{display_name:20}: ✗ Warning - {str(e)[:50]}...")
    
    return import_results, imported_classes

# Run import tests
import_results, fsl_classes = test_fslwrapper_imports()


In [ ]:
# Check FSL environment
def check_fsl_environment():
    """Check if FSL is properly installed and configured"""
    print("Checking FSL Environment:")
    print("=" * 40)
    
    try:
        from fslwrapper import FSLWrapper
        
        # Create wrapper instance
        wrapper = FSLWrapper()
        print("FSL Wrapper initialized successfully!")
        
        # Check environment
        fsl_info = wrapper.check_environment()
        print("\nFSL Environment Information:")
        for key, value in fsl_info.items():
            print(f"  {key}: {value}")
        
        # Get version information
        version_info = wrapper.get_version_info()
        print("\nVersion Information:")
        print(f"  Package Version: {version_info['package_version']}")
        print("  Tool Versions:")
        for tool, version in version_info['tools'].items():
            print(f"    {tool}: {version}")
        
        # List available tools
        tools = wrapper.list_tools()
        print(f"\nAvailable Tools: {', '.join(tools)}")
        
        return wrapper
        
    except Exception as e:
        print(f"FSL environment check failed: {e}")
        print("\nPlease ensure:")
        print("  1. FSL is installed (https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/FslInstallation)")
        print("  2. FSLDIR environment variable is set")
        print("  3. fslpy package is installed (pip install fslpy)")
        return None

# Check FSL environment
if import_results.get("Main Wrapper", False):
    fsl_wrapper = check_fsl_environment()
else:
    print("Skipping FSL environment check due to import failure")
    fsl_wrapper = None


## 2. Complete Demo Pipeline {#demo-pipeline}

Run a complete neuroimaging processing pipeline with real data.


In [ ]:
# Setup sample data directory
from pathlib import Path
import shutil

def setup_sample_data():
    """Setup sample data for the demo"""
    print("Setting up sample data:")
    print("=" * 50)
    
    # Create sample data directory
    sample_dir = Path("sample_data")
    sample_dir.mkdir(exist_ok=True)
    print(f"✓ Sample data directory: {sample_dir.absolute()}")
    
    # Try to find MNI152 template from FSL
    fsl_data_paths = [
        Path("/usr/local/fsl/data/standard"),
        Path("/usr/share/fsl/data/standard"),
        Path.home() / "fsl" / "data" / "standard"
    ]
    
    reference_file = None
    for fsl_path in fsl_data_paths:
        mni_template = fsl_path / "MNI152_T1_2mm.nii.gz"
        if mni_template.exists():
            reference_file = mni_template
            print(f"✓ Found MNI152 template: {reference_file}")
            break
    
    if not reference_file:
        print("⚠ MNI152 template not found in standard FSL locations")
        print("  You can:")
        print("  1. Provide your own T1-weighted image")
        print("  2. Download MNI152 template using: fsl-get_standard MNI152_T1_2mm")
        return None, None
    
    # Create demo input file
    input_file = sample_dir / "demo_input.nii.gz"
    if not input_file.exists():
        shutil.copy2(reference_file, input_file)
        print(f"✓ Created demo input: {input_file.name}")
    else:
        print(f"✓ Demo input exists: {input_file.name}")
    
    print("\n" + "=" * 50)
    return input_file, reference_file

# Setup sample data
if fsl_wrapper is not None:
    input_file, reference_file = setup_sample_data()
else:
    print("FSL Wrapper not available, skipping sample data setup")
    input_file, reference_file = None, None


### Step 1: Brain Extraction


In [ ]:
def run_brain_extraction(fsl_wrapper, input_file):
    """Run brain extraction on the input file"""
    print("STEP 1: Brain Extraction")
    print("=" * 50)
    
    if fsl_wrapper is None or input_file is None:
        print("⚠ Prerequisites not met, skipping...")
        return None
    
    try:
        print(f"Input: {input_file.name}")
        print("Running BET with parameters:")
        print("  fractional_intensity = 0.5")
        print("  robust = True")
        print()
        
        # Run brain extraction
        result = fsl_wrapper.run_brain_extraction(
            input_file=str(input_file),
            output_file=None,  # Auto-generate output filename
            fractional_intensity=0.5,
            robust=True
        )
        
        if result['success']:
            output_file = Path(result['output_file'])
            print("✓ Brain extraction completed!")
            print(f"  Output: {output_file.name}")
            print(f"  Return code: {result['return_code']}")
            return output_file
        else:
            print("✗ Brain extraction failed!")
            print(f"  Error: {result.get('stderr', 'Unknown error')}")
            return None
            
    except Exception as e:
        print(f"✗ Error during brain extraction: {e}")
        return None

# Run brain extraction
brain_file = run_brain_extraction(fsl_wrapper, input_file)
print("\n" + "=" * 50)


### Step 2: Image Registration to MNI Space


In [ ]:
def run_registration(fsl_wrapper, brain_file, reference_file):
    """Run registration to MNI space"""
    print("STEP 2: Image Registration")
    print("=" * 50)
    
    if fsl_wrapper is None or brain_file is None or reference_file is None:
        print("⚠ Prerequisites not met, skipping...")
        return None
    
    try:
        print(f"Input: {brain_file.name}")
        print(f"Reference: {reference_file.name}")
        print("Running FLIRT with parameters:")
        print("  cost_function = mutualinfo")
        print("  dof = 12 (affine)")
        print()
        
        # Run registration
        result = fsl_wrapper.run_registration(
            input_file=str(brain_file),
            reference_file=str(reference_file),
            output_file=None,
            cost_function="mutualinfo",
            dof=12
        )
        
        if result['success']:
            output_file = Path(result['output_file'])
            matrix_file = Path(result.get('output_matrix', 'N/A'))
            print("✓ Registration completed!")
            print(f"  Output image: {output_file.name}")
            print(f"  Transform matrix: {matrix_file.name if matrix_file.exists() else 'N/A'}")
            print(f"  Return code: {result['return_code']}")
            return output_file
        else:
            print("✗ Registration failed!")
            print(f"  Error: {result.get('stderr', 'Unknown error')}")
            return None
            
    except Exception as e:
        print(f"✗ Error during registration: {e}")
        return None

# Run registration
registered_file = run_registration(fsl_wrapper, brain_file, reference_file)
print("\n" + "=" * 50)


### Step 3: Tissue Segmentation


In [ ]:
def run_segmentation(fsl_wrapper, input_file):
    """Run tissue segmentation"""
    print("STEP 3: Tissue Segmentation")
    print("=" * 50)
    
    if fsl_wrapper is None or input_file is None:
        print("⚠ Prerequisites not met, skipping...")
        return None
    
    try:
        print(f"Input: {input_file.name}")
        print("Running FAST with parameters:")
        print("  number_classes = 3 (CSF, GM, WM)")
        print("  bias_correction = True")
        print()
        
        # Run segmentation
        result = fsl_wrapper.run_segmentation(
            input_file=str(input_file),
            output_basename=None,
            number_classes=3,
            bias_correction=True
        )
        
        if result['success']:
            output_files = result.get('output_files', [])
            print("✓ Tissue segmentation completed!")
            print(f"  Generated {len(output_files)} output files:")
            for i, out_file in enumerate(output_files[:5], 1):  # Show first 5
                print(f"    {i}. {Path(out_file).name}")
            if len(output_files) > 5:
                print(f"    ... and {len(output_files) - 5} more")
            print(f"  Return code: {result['return_code']}")
            return output_files
        else:
            print("✗ Segmentation failed!")
            print(f"  Error: {result.get('stderr', 'Unknown error')}")
            return None
            
    except Exception as e:
        print(f"✗ Error during segmentation: {e}")
        return None

# Run segmentation
seg_files = run_segmentation(fsl_wrapper, registered_file)
print("\n" + "=" * 50)


### Step 4: Post-Processing - Create Binary Brain Mask


In [ ]:
def run_post_processing(fsl_wrapper, seg_files):
    """Create binary brain mask from segmentation"""
    print("STEP 4: Post-Processing")
    print("=" * 50)
    
    if fsl_wrapper is None or seg_files is None or len(seg_files) == 0:
        print("⚠ Prerequisites not met, skipping...")
        return None
    
    try:
        # Find the segmentation file (usually ends with _seg.nii.gz)
        seg_file = None
        for f in seg_files:
            if '_seg.nii' in str(f):
                seg_file = Path(f)
                break
        
        if seg_file is None:
            print("⚠ Could not find segmentation file")
            return None
        
        print(f"Input: {seg_file.name}")
        print("Creating binary brain mask:")
        print("  Operations: threshold > 1, then binarize")
        print()
        
        # Create binary mask
        output_file = seg_file.parent / "brain_mask.nii.gz"
        result = fsl_wrapper.run_math_operation(
            input_file=str(seg_file),
            output_file=str(output_file),
            operations=["-thr", "1", "-bin"]
        )
        
        if result['success']:
            print("✓ Binary mask created!")
            print(f"  Output: {output_file.name}")
            print(f"  Return code: {result['return_code']}")
            return output_file
        else:
            print("✗ Mask creation failed!")
            print(f"  Error: {result.get('stderr', 'Unknown error')}")
            return None
            
    except Exception as e:
        print(f"✗ Error during post-processing: {e}")
        return None

# Run post-processing
mask_file = run_post_processing(fsl_wrapper, seg_files)
print("\n" + "=" * 50)


## 3. Pipeline Approach - Run All Steps Automatically {#pipeline}

Instead of running steps individually, you can create a pipeline that runs all steps automatically.


In [ ]:
def run_complete_pipeline(fsl_wrapper, input_file, reference_file):
    """Run complete processing pipeline"""
    print("Running Complete Processing Pipeline:")
    print("=" * 50)
    
    if fsl_wrapper is None or input_file is None:
        print("⚠ Prerequisites not met, skipping...")
        return None
    
    try:
        # Create pipeline
        pipeline = fsl_wrapper.create_pipeline("Complete Brain Processing Pipeline")
        print("✓ Pipeline created")
        
        # Calculate output filenames
        base_name = input_file.stem.replace('.nii', '')
        
        # Add Step 1: Brain Extraction
        pipeline.add_step(
            "Brain Extraction",
            fsl_wrapper.bet,
            {
                'input_file': str(input_file),
                'output_file': None,
                'fractional_intensity': 0.5,
                'robust': True
            }
        )
        print("✓ Added Step 1: Brain Extraction")
        
        # Add Step 2: Registration (if reference available)
        if reference_file:
            pipeline.add_step(
                "MNI Registration",
                fsl_wrapper.flirt,
                {
                    'input_file': str(input_file.parent / f"{base_name}_brain.nii.gz"),
                    'reference_file': str(reference_file),
                    'output_file': None,
                    'cost_function': 'mutualinfo',
                    'dof': 12
                }
            )
            print("✓ Added Step 2: Registration to MNI")
            seg_input = f"{base_name}_brain_registered.nii.gz"
        else:
            seg_input = f"{base_name}_brain.nii.gz"
        
        # Add Step 3: Tissue Segmentation
        pipeline.add_step(
            "Tissue Segmentation",
            fsl_wrapper.fast,
            {
                'input_file': str(input_file.parent / seg_input),
                'output_basename': None,
                'number_classes': 3,
                'bias_correction': True
            }
        )
        print("✓ Added Step 3: Tissue Segmentation")
        
        print("\n" + "=" * 50)
        print("Executing pipeline...")
        print("=" * 50)
        
        # Execute pipeline
        results = pipeline.execute(stop_on_error=True)
        
        print("\n" + "=" * 50)
        print("Pipeline Execution Results:")
        print("=" * 50)
        
        # Check results
        failed_steps = pipeline.get_failed_steps()
        if failed_steps:
            print(f"✗ Pipeline failed with {len(failed_steps)} failed step(s):")
            for step in failed_steps:
                print(f"  • {step['step_name']}: {step['result'].get_error_message()}")
            return False
        else:
            print(f"✓ Pipeline completed successfully!")
            print(f"\nTotal steps executed: {len(results)}")
            for i, result in enumerate(results, 1):
                status = "✓" if result.is_success() else "✗"
                step_name = result.metadata.get('step_name', 'Unknown')
                print(f"  {status} Step {i}: {step_name}")
            return True
            
    except Exception as e:
        print(f"✗ Error during pipeline execution: {e}")
        return False

# Run complete pipeline
print("This demonstrates running all steps as a single pipeline:\n")
pipeline_success = run_complete_pipeline(fsl_wrapper, input_file, reference_file)
print("\n" + "=" * 50)


## Summary

This notebook demonstrates the FSL Wrapper workflow for neuroimaging processing.

1. **Environment Setup**: Checking FSL installation and wrapper availability
2. **Individual Steps**: Running brain extraction, registration, segmentation, and post-processing
3. **Pipeline Approach**: Automating multiple steps in a single workflow
4. **Error Handling**: Proper exception handling for robust processing
5. **Advanced Usage**: Custom parameters, timeouts, and tool access

### Key FSL Tools:

- **BET (Brain Extraction Tool)**: Removes skull and non-brain tissue
- **FLIRT (Linear Registration)**: Registers images to standard space
- **FAST (Segmentation)**: Segments brain tissue types (CSF, GM, WM)
- **FSLMaths**: Mathematical operations on images

### Best Practices:

✓ Always check FSL environment before processing  
✓ Use brain-extracted images for registration  
✓ Apply bias correction before segmentation  
✓ Use appropriate cost functions (mutualinfo for multi-modal)  
✓ Use pipelines for reproducible workflows  
✓ Handle exceptions appropriately  
✓ Set reasonable timeouts for long operations  

### Output Files in sample_data/:

After running this tutorial, you should have:
- `demo_input.nii.gz` - Original input image
- `*_brain.nii.gz` - Brain-extracted image
- `*_registered.nii.gz` - Registered image (if reference available)
- `*_seg.nii.gz` - Tissue segmentation
- `*_pve_*.nii.gz` - Probability maps for tissue classes
- `brain_mask.nii.gz` - Binary brain mask

### Next Steps:

1. Process your own neuroimaging data
2. Customize parameters for your specific needs
3. Build custom pipelines for your analysis
4. Explore additional FSL tools
5. Integrate with other neuroimaging packages

**This is a demo for FSL Wrapper usage. For production use, please:**
- Use real medical imaging data
- Adjust parameters based on your data quality
- Validate outputs visually and quantitatively
- Follow your institution's data processing protocols
